In [ ]:
from src.utils import *
from sklearn import svm
from scipy import sparse
import time
from tqdm import tqdm_notebook
import os

In [ ]:
X, Y = load(k=2), load(X=False, k=2)

In [ ]:
def kernel(X, Z):
    return (X[:, None, :] == Z[None, :, :]).sum(-1)

In [ ]:
clf = svm.SVC(kernel=kernel)
evaluate(clf, X, Y)

## Spectrum Kernel

#### Black magic function to extract subsequences as views of the source array

In [ ]:
def rolling_window(a, window):
    """
    Make an ndarray with a rolling window of the last dimension

    Parameters
    ----------
    a : array_like
       Array to add rolling window to
    window : int
       Size of rolling window

    Returns
    -------
    Array that is a view of the original array with a added dimension
    of size w.

    Examples
    --------
    >>> x=np.arange(10).reshape((2,5))
    >>> rolling_window(x, 3)
    array([[[0, 1, 2], [1, 2, 3], [2, 3, 4]],
          [[5, 6, 7], [6, 7, 8], [7, 8, 9]]])

    Calculate rolling mean of last dimension:
    >>> np.mean(rolling_window(x, 3), -1)
    array([[ 1.,  2.,  3.],
          [ 6.,  7.,  8.]])

    """
    if window < 1:
        raise ValueError("`window` must be at least 1.")
    if window > a.shape[-1]:
        raise ValueError("`window` is too long.")
        
    shape = a.shape[:-1] + (a.shape[-1] - window + 1, window)
    strides = a.strides + (a.strides[-1],)
    
    return np.lib.stride_tricks.as_strided(a, shape=shape, strides=strides)

In [ ]:
def sparse_prod(a, b, n, m):
    inds_a, counts_a = a
    inds_b, counts_b = b
    
    kernel = np.zeros((n, m))
    table_a = dict()
    
    last_i = -1
    xs = []
    counts = []
    
    for (i, x), count in zip(inds_a, counts_a):
        if i != last_i:
            table_a[last_i] = (xs, counts)
            
            xs = []
            counts = []
            last_i = i
        
        xs.append(int(x))
        counts.append(count)
    
    table_a[last_i] = (xs, counts)
    
    for (j, y), count in zip(inds_b, counts_b):
        if j not in table_a:
            continue
        xs, counts = table_a[j]

        kernel[xs, int(y)] += np.asarray(counts)*count
    
    return kernel

In [ ]:
def k_grams(X, k=2, m=4, ret_inds=False):
    N, L = X.shape
    
    encoding = (m ** np.arange(k, dtype=int)).astype(np.uint64)
    
    k_grams_indices = rolling_window(X, window=k).dot(encoding)
    
    xs = np.repeat(np.arange(N), repeats=k_grams_indices.shape[1])
    ys = k_grams_indices.reshape(-1)
    
    inds = np.stack((xs, ys), axis=1)
    inds, counts = np.unique(inds, axis=0, return_counts=True)
    
    res = sparse.csr_matrix((counts, (inds[:, 0], inds[:, 1])), shape=(N, m ** k), dtype=int)
    if ret_inds:
        return res, np.unique(np.stack((ys, xs), axis=1), axis=0, return_counts=True)
    return res


def k_spectrum(X, Y=None, k=2, m=4):
    """
     Computes the k-spectrum kernel between the sequences from X and Y.
     If Y is None, computes the k-spectrum between sequences from X.
     
     NB: 'normalizes' the kernel, but does not center it.
    """
    if k > 13:
        return k_spectrum_extreme(X, Y, k, m)
    
    k_grams_X = k_grams(X, k=k, m=m)
    k_grams_Y = k_grams_X if Y is None else k_grams(Y, k=k, m=m)
    
    # TODO: Normalization? Centering?
    norms_X = sparse.linalg.norm(k_grams_X, axis=1)
    norms_Y = sparse.linalg.norm(k_grams_Y, axis=1)
    
    K = k_grams_X.dot(k_grams_Y.T).toarray() / np.outer(norms_X, norms_Y)
    
    return K


def k_spectrum_extreme(X, Y=None, k=2, m=4):
    k_grams_X, k_grams_inds_X = k_grams(X, k=k, m=m, ret_inds=True)
    k_grams_Y, k_grams_inds_Y = (k_grams_X, k_grams_inds_X) if Y is None else k_grams(Y, k=k, m=m, ret_inds=True)
    
    K = sparse_prod(k_grams_inds_X, k_grams_inds_Y, len(X), len(X) if Y is None else len(Y))
    
    # TODO: Normalization? Centering?
    norms_X = sparse.linalg.norm(k_grams_X, axis=1)
    norms_Y = sparse.linalg.norm(k_grams_Y, axis=1)
    
    K /= np.outer(norms_X, norms_Y)
    
    return K

def cum_spectrum(X, Y=None, k=5, tqdm=False):
    """
     Computes the sum of the spectrum kernels between X and Y, up to k.
    """
    shape = (len(X), len(X) if Y is None else len(Y))
    kernel = np.zeros(shape)
    
    for i in (tqdm_notebook(range(1, k+1)) if tqdm else range(1, k+1)):
        kernel += k_spectrum(X, Y=Y, k=i)
    
    return kernel / k

## Evaluation functions

### Cross validation

In [ ]:
def k_folds_indices(n, k):
    """
     Return k pairs (train_inds, valid_inds) of arrays containing the training and validation indices for each split.
    """
    assert(n % k == 0)
    m = n // k
    indices = np.random.permutation(n)
    
    folds = []
    for i in range(k):
        folds.append((np.concatenate((indices[:i*m], indices[(i+1)*m:])),
                      indices[i*m:(i+1)*m]))
    return folds

In [ ]:
def evaluate(classifier, K, Y, folds=5):
    """
    :param classifier: classifier to evaluate
    :param K: precomputed kernel matrix of shape (n_samples, n_samples)
    :param Y: training labels of shape (n_samples, )
    :param folds: number of folds to use
    
    Evaluates the classifier using cross-validation.
    Returns the mean and std of the validation and train scores.:
        (valid_scores_mean, valid_scores_std, train_scores_mean, train_scores_std)
    """
    
    N = len(K)
    valid_scores = []
    train_scores = []
    for train_inds, valid_inds in k_folds_indices(N, folds):
        classifier.fit(K[np.ix_(train_inds, train_inds)], Y[train_inds])
        
        valid_scores.append((classifier.predict(K[np.ix_(valid_inds, train_inds)]) == Y[valid_inds]).mean())
        train_scores.append((classifier.predict(K[np.ix_(train_inds, train_inds)]) == Y[train_inds]).mean())
        
    valid_scores = np.array(valid_scores)
    train_scores = np.array(train_scores)
    
    return valid_scores.mean(), valid_scores.std(), train_scores.mean(), train_scores.std()

In [ ]:
def global_evaluate(classifier, Ks, Ys, Cs, folds=5, **params):
    """
    :param classifier: classifier to evaluate
    :param Ks: list precomputed kernel matrices of shape list(n_samples_i, n_samples_i)
    :param Ys: training labels of shape list(n_samples_i,)
    :param Cs: list of parameters C
    :param folds: number of folds to use
    :param params: additional parameters to instantiate the classifier
    
    Evaluates a classifier on several data sets, and averages the results.
    """
    return np.mean(np.array([evaluate(classifier(C=C, **params), K, Y, folds) for (K, Y, C) in zip(Ks, Ys, Cs)]), axis=0)

### Grid Search

In [ ]:
def grid_search(K, Y, folds=5, C_min=-7, C_max=6):
    """
     Grid search on C values, from 10^C_min to 10^C_max, using k-fold cross validation.
    """
    results = []
    for C in 10. ** np.arange(C_min, C_max + 1):
        results.append(np.array(evaluate(svm.SVC(kernel='precomputed', C=C), K, Y, folds=folds)))
        print('%.0e \t ' % C,
              'Validation %.2f%% +- %.2f\t Train %.2f%%\t +- %.2f' %
              tuple(100 * results[-1]))
    
    return 10. ** np.arange(C_min, C_max + 1), results

### Test predictions

In [ ]:
def test(model, train_Xs, train_Ys, test_Xs, Cs, file_name='predictions', **params):
    """
    For each data set, trains the model on the train data, then computes predictions on the test data.
    Saves the predictions to the specified file.
    
    Also evaluates the performance of the specified model using cross-validation on the train data.
    
    NB: train_Xs and test_Xs can be either precomputed kernel matrices, or simply the train and test features,
    depending on the model and parameters.
    """
    # Evaluation using k-fold validation.
    print('Evaluation on train data (using cross validation)')
    print('Validation %.2f\%% +- %.2f\t Train %.2f%%\t +- %.2f' %
          tuple(100 * global_evaluate(model, train_Xs, train_Ys, Cs, **params)))
    predictions = []
    for k, (X_train, Y_train, X_test, C) in enumerate(zip(train_Xs, train_Ys, test_Xs, Cs)):
        model_k = model(C=C, **params)
        model_k.fit(X_train, Y_train)
        
        predictions.append(model_k.predict(X_test))
    
    predictions = np.concatenate(predictions)
    
    predictions_dir = 'predictions'
    if not os.path.isdir(predictions_dir):
        os.mkdir(predictions_dir)
    
    np.savetxt('%s/%s.csv' % (predictions_dir, file_name),
               np.stack((np.arange(len(predictions)),
                         predictions), axis=1),
               header='Id,Bound', comments='', fmt='%d', delimiter=',')
    return predictions

## Baseline using Spectrum kernel

The computations are slowed down over $k=13$.

In [ ]:
spectrum_k = 12

### Data loading

In [ ]:
train_Xs = [load(k=k) for k in range(3)]
train_Ys = [load(X=False, k=k) for k in range(3)]
test_Xs = [load(k=k, train=False) for k in range(3)]

### Kernel Computing

In [ ]:
start = time.time()
tqdm = True

train_Ks = [cum_spectrum(train_X, k=spectrum_k, tqdm=tqdm) for train_X in train_Xs]
test_Ks  = [cum_spectrum(test_X, train_X, k=spectrum_k, tqdm=tqdm) for (test_X, train_X) in zip(test_Xs, train_Xs)]

print("Total Duration: %.1f seconds." % (time.time() - start))

### Grid-search to find the best values for parameter C

#### Dataset 0

In [ ]:
gs0 = grid_search(train_Ks[0], train_Ys[0])

#### Dataset 1

In [ ]:
gs1 = grid_search(train_Ks[1], train_Ys[1])

#### Dataset 2

In [ ]:
gs2 = grid_search(train_Ks[2], train_Ys[2])

#### Best values

In [ ]:
greedy_Cs = [Cs[np.argmax(np.array(scores)[:, 0] - np.array(scores)[:, 1])]
             for (Cs, scores) in [gs0, gs1, gs2]]  # Values with best validation score.

Cs = (1, 1, 1)               # Values without strong overfitting (i.e. perfect training accuracy).

In [ ]:
print(greedy_Cs)

### Generating test predictions

#### Greedy values of C

In [ ]:
_ = test(svm.SVC,
         train_Ks, train_Ys, test_Ks,
         Cs=greedy_Cs, kernel='precomputed',
         file_name='spectrum_greedy_predictions_%d' % spectrum_k)

#### Soft values of C

In [ ]:
_ = test(svm.SVC,
         train_Ks, train_Ys, test_Ks,
         Cs=Cs, kernel='precomputed',
         file_name='spectrum_soft_predictions_%d' % spectrum_k)